<font size="6">**Task 1: data mining**</font>

**1.1 Get pokemon data from pokeAPI**

In [1]:
import requests
import json
import os

# 定义保存图片的目录
save_directory = "data/pokemon_images"
os.makedirs(save_directory, exist_ok=True)

# 获取所有宝可梦的基本信息
url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
response = requests.get(url)
data = response.json()

# 创建一个字典来存储宝可梦的名称和属性
pokemon_info = {}

# 遍历每个宝可梦
for pokemon in data['results']:
    pokemon_name = pokemon['name']
    pokemon_url = pokemon['url']
    response = requests.get(pokemon_url)
    pokemon_data = response.json()

    # 获取宝可梦的属性
    types = pokemon_data['types']
    type_names = [type_info['type']['name'] for type_info in types]

    # 获取宝可梦的图片链接
    pic1_url = pokemon_data['sprites']['other']['official-artwork']['front_default']
    pic2_url = pokemon_data['sprites']['other']['home']['front_default']
    pic3_url = pokemon_data['sprites']['front_default']

    # 下载并保存图片
    for i, pic_url in enumerate([pic1_url, pic2_url, pic3_url], start=1):
        if pic_url:
            img_response = requests.get(pic_url)
            if img_response.status_code == 200:
                img_name = f"{pokemon_name}_pic{i}.png"
                img_path = os.path.join(save_directory, img_name)
                with open(img_path, 'wb') as file:
                    file.write(img_response.content)

    # 将宝可梦的名称和属性存储到字典中
    pokemon_info[pokemon_name] = {
        "types": type_names,
        "images": [os.path.join(save_directory, f"{pokemon_name}_pic{i}.png") for i in range(1, 4) if eval(f"pic{i}_url")]
    }

# 将字典转换为 JSON 格式并保存到文件中
with open('data/pokemon_info.json', 'w') as json_file:
    json.dump(pokemon_info, json_file, indent=4)

print(f"所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 {save_directory} 目录中。")

所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 data/pokemon_images 目录中。


<font size="6">**Task 2: Training of pokemon type**</font>

**2.1 Preprocessing**

In [26]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

# 定义自定义数据集类
class PokemonDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.image_files = []
        self.labels = []
        
        # 遍历数据字典，获取所有图像文件路径和对应的标签（属性）
        for pokemon_name, info in data_dict.items():
            for img_path in info['images']:
                self.image_files.append(img_path)
                self.labels.append(info['types'])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        
        # 获取宝可梦的标签（属性）
        labels = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # 将字符串标签转换为整数标签
        label_map = {'grass': 0, 'poison': 1, 'fire': 2, 'water': 3, 'electric': 4, 'ice': 5, 'fighting': 6,
                     'ground': 7, 'flying': 8, 'psychic': 9, 'bug': 10, 'rock': 11, 'ghost': 12, 'dark': 13,
                     'dragon': 14, 'steel': 15, 'fairy': 16, 'normal': 17}
        labels = [label_map[label] for label in labels]
        
        # 将标签转换为多标签二进制格式
        multi_label = torch.zeros(len(label_map), dtype=torch.float32)
        for label in labels:
            multi_label[label] = 1.0
        
        return image, multi_label

# 读取数据字典
with open('data/pokemon_info.json', 'r') as f:
    data_dict = json.load(f)

# 定义训练数据的转换
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# 创建数据集
pokemon_dataset = PokemonDataset(data_dict=data_dict, transform=data_transforms)

# 将数据集分割为训练集、验证集和测试集（80%、10%、10%）
train_size = int(0.8 * len(pokemon_dataset))
val_size = int(0.1 * len(pokemon_dataset))
test_size = len(pokemon_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(pokemon_dataset, [train_size, val_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


**2.2 Model and training**

In [28]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 定义自定义数据集类
class PokemonDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.image_files = []
        self.labels = []
        
        # 遍历数据字典，获取所有图像文件路径和对应的标签（属性）
        for pokemon_name, info in data_dict.items():
            for img_path in info['images']:
                self.image_files.append(img_path)
                self.labels.append(info['types'])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        
        # 使用OpenCV加载图像
        image = cv2.imread(img_path)
        if image is None:
            print(f"UnidentifiedImageError: cannot identify image file '{img_path}'")
            return self.__getitem__((idx + 1) % len(self.image_files))
        
        # 将图像从BGR转换为RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 获取宝可梦的标签（属性）
        labels = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # 将字符串标签转换为整数标签
        label_map = {'grass': 0, 'poison': 1, 'fire': 2, 'water': 3, 'electric': 4, 'ice': 5, 'fighting': 6,
                     'ground': 7, 'flying': 8, 'psychic': 9, 'bug': 10, 'rock': 11, 'ghost': 12, 'dark': 13,
                     'dragon': 14, 'steel': 15, 'fairy': 16, 'normal': 17}
        labels = [label_map[label] for label in labels]
        
        # 将标签转换为多标签二进制格式
        multi_label = torch.zeros(len(label_map), dtype=torch.float32)
        for label in labels:
            multi_label[label] = 1.0
        
        return image, multi_label

# 定义函数来提取图像特征
def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
    return np.concatenate(features)


# 读取数据字典
with open('data/pokemon_info.json', 'r') as f:
    data_dict = json.load(f)

# 定义训练数据的转换
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# 创建数据集
pokemon_dataset = PokemonDataset(data_dict=data_dict, transform=data_transforms)

# 将数据集分割为训练集、验证集和测试集（80%、10%、10%）
train_size = int(0.8 * len(pokemon_dataset))
val_size = int(0.1 * len(pokemon_dataset))
test_size = len(pokemon_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(pokemon_dataset, [train_size, val_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 检查是否有可用的GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载预训练的ResNet18模型
model = models.resnet18(pretrained=True)

# 修改最后一层以匹配属性的数量（假设有18种可能的属性）
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

# 将模型移动到GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练循环
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs_labels in train_loader:
        if inputs_labels is None:
            continue
        
        inputs, labels = inputs_labels
        
        # 将输入和标签移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 验证循环
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs_labels in val_loader:
            if inputs_labels is None:
                continue
            
            inputs, labels = inputs_labels
            
            # 将输入和标签移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

print("Training complete.")

# 测试循环（可选）
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs_labels in test_loader:
        if inputs_labels is None:
            continue
        
        inputs, labels = inputs_labels
        
        # 将输入和标签移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")



Using device: cuda:0
Epoch 1/20, Loss: 0.3015347325624271
Validation Loss: 0.28118698222121014
Epoch 2/20, Loss: 0.2769729947487925
Validation Loss: 0.26360021900270403
Epoch 3/20, Loss: 0.2636679448959432
Validation Loss: 0.2527120541205111
Epoch 4/20, Loss: 0.25308770371765055
Validation Loss: 0.2441973531829942
Epoch 5/20, Loss: 0.24287912857578825
Validation Loss: 0.2363066161723481
Epoch 6/20, Loss: 0.23445101987589825
Validation Loss: 0.2303259541693422
Epoch 7/20, Loss: 0.22668955533963103
Validation Loss: 0.22565625984336912
Epoch 8/20, Loss: 0.21940744281239768
Validation Loss: 0.2215224751799377
Epoch 9/20, Loss: 0.20787746508465837
Validation Loss: 0.21708014538300405
Epoch 10/20, Loss: 0.19980790511429927
Validation Loss: 0.2146109573005401
Epoch 11/20, Loss: 0.19193015428057297
Validation Loss: 0.21295362611099616
Epoch 12/20, Loss: 0.1812104575741322
Validation Loss: 0.21135934019826122
Epoch 13/20, Loss: 0.17058763386672024
Validation Loss: 0.20369247996161893
Epoch 14/2

<font size="6">**Task 3: Analysis of types from other games**</font>

<font size="6">**Task 4: Most similar pokemon**</font>

**4.1 Different functions of calculation of similarity**

4.1.1 cosine_similarity

In [29]:
# 示例：查找与查询图像最相似的三个宝可梦图像
# 定义函数来找到最相似的宝可梦图像
def find_similar_images(query_image_path, model, feature_extractor, dataset, dataloader, top_k=3):
    # 加载查询图像并进行预处理
    query_image = cv2.imread(query_image_path)
    if query_image is None:
        raise ValueError(f"Cannot identify image file '{query_image_path}'")
    
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = data_transforms(query_image).unsqueeze(0).to(device)
    
    # 提取查询图像的特征
    with torch.no_grad():
        query_features = feature_extractor(query_image).cpu().numpy()
    
    # 提取数据集中所有图像的特征
    dataset_features = extract_features(feature_extractor, dataloader)
    
    # 计算余弦相似度
    similarities = cosine_similarity(query_features, dataset_features)
    
    # 找到最相似的图像索引
    top_k_indices = np.argsort(similarities[0])[::-1][:top_k]
    
    # 返回最相似的图像路径和相似度分数
    similar_images = [(dataset.image_files[idx], similarities[0][idx]) for idx in top_k_indices]
    
    return similar_images

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images(query_image_path, model, model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\omanyte_pic1.png, Similarity Score: 0.9607487916946411
Image: data/pokemon_images\roserade_pic2.png, Similarity Score: 0.9476977586746216
Image: data/pokemon_images\fraxure_pic1.png, Similarity Score: 0.9373520612716675


4.1.2 Euclidean

In [36]:
from sklearn.metrics.pairwise import euclidean_distances

def find_similar_images_euclidean(query_image_path, model, feature_extractor, dataset, dataloader, top_k=3):
    query_image = cv2.imread(query_image_path)
    if query_image is None:
        raise ValueError(f"Cannot identify image file '{query_image_path}'")

    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = data_transforms(query_image).unsqueeze(0).to(device)

    with torch.no_grad():
        query_features = feature_extractor(query_image).cpu().numpy()

    dataset_features = extract_features(feature_extractor, dataloader)

    distances = euclidean_distances(query_features, dataset_features)

    top_k_indices = np.argsort(distances[0])[:top_k]

    similar_images = [(dataset.image_files[idx], distances[0][idx]) for idx in top_k_indices]

    return similar_images

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images_euclidean(query_image_path, model, model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Distance: {score}")

Most similar images:
Image: data/pokemon_images\nidoqueen_pic2.png, Distance: 9.441254615783691
Image: data/pokemon_images\toxapex_pic2.png, Distance: 9.872849464416504
Image: data/pokemon_images\kirlia_pic3.png, Distance: 10.014799118041992


4.1.3 VGG

In [31]:
from torchvision.models import vgg16

# 加载预训练的VGG16模型
vgg_model = vgg16(pretrained=True)
vgg_model.classifier = nn.Sequential(*list(vgg_model.classifier.children())[:-1])  # 移除最后一层
vgg_model = vgg_model.to(device)

def extract_vgg_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
    return np.concatenate(features)

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images(query_image_path, vgg_model, vgg_model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

C:\Users\业\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\业\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Most similar images:
Image: data/pokemon_images\inkay_pic3.png, Similarity Score: 0.46852004528045654
Image: data/pokemon_images\simipour_pic1.png, Similarity Score: 0.4677070379257202
Image: data/pokemon_images\koraidon_pic2.png, Similarity Score: 0.45726901292800903


4.1.4 Hashes

In [32]:
import cv2
import imagehash
import numpy as np
 
def find_similar_images_hash(query_image_path, dataset, top_k=3):
    # 使用 OpenCV 读取图像
    query_image = cv2.imread(query_image_path)
    # OpenCV 读取的图像是 BGR 格式，需要转换为 RGB 格式
    query_image_rgb = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    # 将图像转换为 Pillow 图像对象以使用 imagehash 库
    query_image_pil = Image.fromarray(query_image_rgb)
    query_hash = imagehash.phash(query_image_pil)
 
    image_hashes = []
    for img_path in dataset.image_files:
        # 使用 OpenCV 读取图像
        img = cv2.imread(img_path)
        # 转换为 RGB 格式
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # 转换为 Pillow 图像对象
        img_pil = Image.fromarray(img_rgb)
        # 计算图像哈希
        img_hash = imagehash.phash(img_pil)
        image_hashes.append((img_path, img_hash))
 
    similarities = [(img_path, 1 - (query_hash - img_hash) / len(query_hash.hash)**2) for img_path, img_hash in image_hashes]
    similarities.sort(key=lambda x: x[1], reverse=True)
 
    return similarities[:top_k]
 
query_image_path = 'data/palworld_images/Anubis.png'
# 假设 pokemon_dataset 是一个具有 image_files 属性的对象，该属性包含图像文件路径的列表
# 注意：你需要确保 pokemon_dataset 已经被正确定义并包含有效的图像文件路径
similar_images = find_similar_images_hash(query_image_path, pokemon_dataset, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\pichu_pic3.png, Similarity Score: 0.75
Image: data/pokemon_images\chinchou_pic3.png, Similarity Score: 0.71875
Image: data/pokemon_images\porygon2_pic2.png, Similarity Score: 0.71875


4.1.5 Combined

In [42]:
import numpy as np
import cv2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import imagehash

def find_similar_images_combined(query_image_path, model, feature_extractor, dataset, dataloader, top_k=3):
    # 加载查询图像并进行预处理
    query_image = cv2.imread(query_image_path)
    if query_image is None:
        raise ValueError(f"Cannot identify image file '{query_image_path}'")
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image_tensor = data_transforms(query_image).unsqueeze(0).to(device)
    
    # 提取查询图像的特征
    with torch.no_grad():
        query_features = feature_extractor(query_image_tensor).cpu().numpy()
    
    # 提取数据集中所有图像的特征
    dataset_features = extract_features(feature_extractor, dataloader)
    
    # 计算余弦相似度
    cosine_similarities = cosine_similarity(query_features, dataset_features)
    
    # 计算哈希相似度
    query_image_pil = Image.fromarray(query_image)
    query_hash = imagehash.phash(query_image_pil)
    image_hashes = []
    for img_path in dataset.image_files:
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_hash = imagehash.phash(img_pil)
        image_hashes.append((img_path, img_hash))
    
    # 计算哈希相似度的归一化分数（这里使用汉明距离的平方的倒数进行归一化）
    # 注意：由于phash产生的是64位的哈希值，因此最大可能的汉明距离是64
    max_hamming_distance = 64
    hash_similarities = []
    for img_path, img_hash in image_hashes:
        hamming_distance = query_hash - img_hash  # imagehash库支持直接相减得到汉明距离
        normalized_hash_similarity = 1 - (hamming_distance / max_hamming_distance)  # 归一化到[0, 1]
        hash_similarities.append((img_path, normalized_hash_similarity))
    
    # 创建一个字典来存储所有图像的路径和它们的加权相似度分数
    final_scores = {}
    for i, (img_path_cosine, cosine_score) in enumerate(zip(dataset.image_files, cosine_similarities[0])):
        # 由于hash_similarities也是按照dataset.image_files的顺序，因此可以直接用i索引
        img_path_hash, hash_score = hash_similarities[i]
        # 确保两者对应的是同一张图像（理论上应该是，但为了安全起见还是检查一下）
        assert img_path_cosine == img_path_hash, "Image paths mismatch between cosine and hash similarities"
        # 计算加权相似度分数
        final_score = 0.8 * cosine_score + 0.2 * hash_score
        final_scores[img_path_cosine] = final_score
    
    # 根据最终相似度分数排序，返回前三名
    sorted_final_scores = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_final_scores[:top_k]

query_image_path = 'data/palworld_images/Anubis.png'
# 假设 pokemon_dataset 是一个具有 image_files 属性的对象，该属性包含图像文件路径的列表
# 注意：你需要确保 pokemon_dataset 已经被正确定义并包含有效的图像文件路径
similar_images = find_similar_images_combined(query_image_path, model, model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\whirlipede_pic3.png, Similarity Score: 0.8699132442474367
Image: data/pokemon_images\rellor_pic3.png, Similarity Score: 0.8518013000488283
Image: data/pokemon_images\caterpie_pic3.png, Similarity Score: 0.8471257209777833


**4.2  Try to use unet model**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(n_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, n_classes, kernel_size=3, padding=1),
        )
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # 全局平均池化

    def forward(self, x):
        enc = self.encoder(x)
        mid = self.middle(enc)
        dec = self.decoder(mid)
        out = self.global_avg_pool(dec)  # 形状变为 [batch_size, n_classes, 1, 1]
        return out.view(out.size(0), -1)  # 形状变为 [batch_size, n_classes]


# 初始化 UNet 模型
n_channels = 3  # 输入图像通道数（RGB 图像）
n_classes = 18  # 输出类别数（多标签分类）
model_u = UNet(n_channels, n_classes)

# 检查是否有可用的 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model_u = model_u.to(device)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()  # 多标签分类任务的损失函数
optimizer = optim.Adam(model_u.parameters(), lr=0.001)  # 使用 Adam 优化器

Using device: cuda:0


In [19]:
from PIL import Image, ImageFile, UnidentifiedImageError
ImageFile.LOAD_TRUNCATED_IMAGES = True

num_epochs = 40
for epoch in range(num_epochs):
    model_u.train()
    running_loss = 0.0
    for inputs_labels in train_loader:
        if inputs_labels is None:
            continue
        
        inputs, labels = inputs_labels
        
        # 将输入和标签移动到 GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model_u(inputs)

        # 将输出调整为与标签形状一致
        outputs = outputs.view(outputs.size(0), -1)
        labels = labels.view(labels.size(0), -1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 验证循环
    model_u.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs_labels in val_loader:
            if inputs_labels is None:
                continue

            inputs, labels = inputs_labels

            # 将输入和标签移动到 GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model_u(inputs)

            # 将输出调整为与标签形状一致
            outputs = outputs.view(outputs.size(0), -1)
            labels = labels.view(labels.size(0), -1)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Validation Loss: {val_loss/len(val_loader)}")

print("Training complete.")


Epoch 1/40, Loss: 0.29572675599103765
Validation Loss: 0.2938861498205932
Epoch 2/40, Loss: 0.29342376415670846
Validation Loss: 0.29187320940887806
Epoch 3/40, Loss: 0.29265110920439114
Validation Loss: 0.29155574892599556
Epoch 4/40, Loss: 0.29261692968520475
Validation Loss: 0.2922781739652771
Epoch 5/40, Loss: 0.29270499404230266
Validation Loss: 0.29135680367651673
Epoch 6/40, Loss: 0.2923475724350603
Validation Loss: 0.29023011382093133
Epoch 7/40, Loss: 0.29211469066034945
Validation Loss: 0.2907717083532786
Epoch 8/40, Loss: 0.29219488403880534
Validation Loss: 0.2894417121238315
Epoch 9/40, Loss: 0.2914232803711311
Validation Loss: 0.28792982202829775
Epoch 10/40, Loss: 0.2907567130006039
Validation Loss: 0.288155108075781
Epoch 11/40, Loss: 0.29009782461150324
Validation Loss: 0.28611178988033964
Epoch 12/40, Loss: 0.2889221167201514
Validation Loss: 0.28576364544863553
Epoch 13/40, Loss: 0.2884591786586559
Validation Loss: 0.28445007982327764
Epoch 14/40, Loss: 0.28732194460

In [43]:
query_image_path = 'data/palworld_images/Anubis.png'
# 假设 pokemon_dataset 是一个具有 image_files 属性的对象，该属性包含图像文件路径的列表
# 注意：你需要确保 pokemon_dataset 已经被正确定义并包含有效的图像文件路径
similar_images = find_similar_images_combined(query_image_path, model_u, model_u, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\tsareena_pic1.png, Similarity Score: 0.7474530220031739
Image: data/pokemon_images\meditite_pic1.png, Similarity Score: 0.7121516704559326
Image: data/pokemon_images\croconaw_pic3.png, Similarity Score: 0.7039288520812989
